In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/swiggy-restuarant-dataset/swiggy.csv


In [37]:
# Importing the required libraries 
import pandas as pd 
import numpy as np 

# Read the data from the csv 
df_restaurants = pd.read_csv("/kaggle/input/swiggy-restuarant-dataset/swiggy.csv")
df_restaurants.head()

,ID,Area,City,Restaurant,Price,Avg ratings,Total ratings,Food type,Address,Delivery time
0,211,Koramangala,Bangalore,Tandoor Hut,300.0,4.4,100,"Biryani,Chinese,North Indian,South Indian",5Th Block,59
1,221,Koramangala,Bangalore,Tunday Kababi,300.0,4.1,100,"Mughlai,Lucknowi",5Th Block,56
2,246,Jogupalya,Bangalore,Kim Lee,650.0,4.4,100,Chinese,Double Road,50
3,248,Indiranagar,Bangalore,New Punjabi Hotel,250.0,3.9,500,"North Indian,Punjabi,Tandoor,Chinese",80 Feet Road,57
4,249,Indiranagar,Bangalore,Nh8,350.0,4.0,50,"Rajasthani,Gujarati,North Indian,Snacks,Desser...",80 Feet Road,63


In [38]:
df_restaurants['Food type']

0               Biryani,Chinese,North Indian,South Indian
1                                        Mughlai,Lucknowi
2                                                 Chinese
3                    North Indian,Punjabi,Tandoor,Chinese
4       Rajasthani,Gujarati,North Indian,Snacks,Desser...
                              ...                        
8675                                               Pizzas
8676                                         South Indian
8677                               Chinese,Snacks,Tandoor
8678                           North Indian,Indian,Snacks
8679                                 Chaat,Snacks,Chinese
Name: Food type, Length: 8680, dtype: object

In [39]:
# Import the required libraries
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [40]:
# Usinf TfidVectorizer for reducing the stop words in order to make it easier to reduce the data into vectors
# Create the basic matrix for this 
tfidf = TfidfVectorizer(stop_words="english")
df_restaurants['Food type'] = df_restaurants['Food type'].fillna("")
tfidf_matrix = tfidf.fit_transform(df_restaurants['Food type'])

In [41]:
# Cosine_sim - measures the similarity between two vectors of an inner product space
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [42]:
df_restaurants['Restaurant']

0                    Tandoor Hut
1                  Tunday Kababi
2                        Kim Lee
3              New Punjabi Hotel
4                            Nh8
                  ...           
8675                  Malt Pizza
8676    Jay Mata Ji Home Kitchen
8677        Chinese Kitchen King
8678      Shree Ram Paratha Wala
8679                Sassy Street
Name: Restaurant, Length: 8680, dtype: object

In [44]:
# Develop the indices 
indices = pd.Series(df_restaurants.index, index = df_restaurants['Restaurant']).drop_duplicates()
indices

Restaurant
Tandoor Hut                    0
Tunday Kababi                  1
Kim Lee                        2
New Punjabi Hotel              3
Nh8                            4
                            ... 
Malt Pizza                  8675
Jay Mata Ji Home Kitchen    8676
Chinese Kitchen King        8677
Shree Ram Paratha Wala      8678
Sassy Street                8679
Length: 8680, dtype: int64

In [45]:
indices['Tunday Kababi']

1

In [53]:
def get_recommendations(Restaurant, cosine_sim=cosine_sim):
    idx = indices[Restaurant]  # Assuming you have a dictionary called 'indices' that maps restaurant names to their indices
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Get the top 10 most similar restaurants
    sim_index = [i[0] for i in sim_scores]
    
    # Print or return the recommended restaurants
    recommended_restaurants = df_restaurants['Restaurant'].iloc[sim_index].tolist()
    return recommended_restaurants

recommended_restaurants = get_recommendations('Tunday Kababi')
print(recommended_restaurants)


['Tandoori Combo Co', 'Tandoori Dhaba', 'Global Chaska', 'The Biryani Life', 'The Biryani Life', 'The Biryani Life', 'The Biryani Life', 'The Biryani Life', 'The Biryani Life', 'Biryani By Kilo']
